In [1]:
!nvidia-smi

Wed Aug 16 11:10:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 528.49       Driver Version: 528.49       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  Off |
| 31%   30C    P8    17W / 450W |    872MiB / 24564MiB |     20%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Requirements

In [2]:
# !pip install tensorboard==1.14.0
# !pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117
# !pip install pytorch_lightning
# !pip install tensorboard
# !pip install numpy

## Import the libraries

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import random_split
import os
import numpy as np

## The Autoencoder model

In [4]:
class Autoencoder(pl.LightningModule):

    def __init__(self, loss_fn=F.mse_loss, lr=1e-3):
        super(Autoencoder, self).__init__()

        self.loss_fn = loss_fn
        self.lr = lr
        self.steps_per_log_image = 50  # log images every 50 steps

        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=4, stride=2, padding=1),  # [128, 32, 128, 227]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),  # [128, 64, 64, 113]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # [128, 128, 32, 56]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 8, kernel_size=1, stride=1, padding=0)   # [128, 8, 32, 56]
        )

        self.decoder = nn.Sequential(
            nn.Conv2d(8, 128, kernel_size=1, stride=1, padding=0),   # [128, 128, 32, 56]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),  # [128, 128, 64, 112]
            nn.Conv2d(128, 64, kernel_size=3, padding=1),   # [128, 64, 64, 112]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),  # [128, 64, 128, 224]
            nn.Conv2d(64, 32, kernel_size=3, padding=1),   # [128, 32, 128, 224]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Upsample(size=(256, 454), mode='bilinear', align_corners=True),  # [128, 32, 256, 454]
            nn.Conv2d(32, 1, kernel_size=3, padding=1),   # [128, 1, 256, 454]
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def training_step(self, batch, batch_idx):
        x, _ = batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = self.loss_fn(x_hat, x)
        self.log('train_loss', loss)

        return loss

    def validation_step(self, batch, batch_idx):
        x, _ = batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = self.loss_fn(x_hat, x)
        self.log('val_loss', loss)
        return {'val_loss': loss, 'x': x, 'x_hat': x_hat}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        self.log('avg_val_loss', avg_loss)

        # Take one batch of images from the validation set for logging
        x = outputs[0]['x']
        x_hat = outputs[0]['x_hat']
        self.log_images(x, x_hat, self.current_epoch)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=5, verbose=True)
        return {
            'optimizer': optimizer,
            # 'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

    @torch.no_grad()
    def log_images(self, originals, recons, epoch):
        # Denormalize and clamp the images in one step
        originals = torch.clamp(originals * 0.5 + 0.5, 0, 1)
        recons = torch.clamp(recons * 0.5 + 0.5, 0, 1)

        # Select 4 random indices from the batch
        indices = torch.randperm(originals.size(0))[:4]

        # Select 4 random samples from the originals and recons tensors
        originals = originals[indices]
        recons = recons[indices]

        # Create the grid of images
        grid = torchvision.utils.make_grid(originals)
        self.logger.experiment.add_image('Original Images', grid, epoch)

        grid = torchvision.utils.make_grid(recons)
        self.logger.experiment.add_image('Reconstructed Images', grid, epoch)

        log_dir = self.logger.log_dir + "/images"
        os.makedirs(log_dir, exist_ok=True)
        torchvision.utils.save_image(
            recons,
            f"{log_dir}/recons_{epoch}.png",
            nrow=int(np.sqrt(recons.shape[0])),
            padding=0
        )

## Load the dataset

In [5]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Grayscale(),
    # transforms.Resize((256, 455)),
    transforms.CenterCrop((256, 454)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Replace `your_data_dir` with your actual directory containing images.
dataset = datasets.ImageFolder(root='data/encoder_dataset', transform=transform)
# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=8)
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=8)

torch.set_float32_matmul_precision('medium')

## Train the model

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

# set the model_name and log_name
model_name = "Autoencoder"
log_name = "Experiment1"

# set to true if resuming from checkpoint
resume = False
checkpoint_path = f"{model_name}-{log_name}/checkpoints/last-v1.ckpt"

# setup logger
logger = TensorBoardLogger("logs", name=f"{model_name}-{log_name}")

# setup model checkpointing
checkpoint_callback = ModelCheckpoint(
    dirpath=f"logs/{model_name}-{log_name}/checkpoints",
    every_n_epochs=1,
    save_last=True,
    save_top_k=3,
    monitor='val_loss',
    mode='min'
)

# define the model
model = Autoencoder()

# setup trainer
ae_trainer = pl.Trainer(
    precision=16,
    callbacks=[checkpoint_callback],
    default_root_dir=f"logs/{model_name}-{log_name}/checkpoints",
    logger=logger,
    accelerator="gpu",
    log_every_n_steps=10,
    max_epochs=1000
)

# fit the model
checkpoint_path = None if not resume else f"logs/{checkpoint_path}"
ae_trainer.fit(model, train_dataloader, val_dataloader,ckpt_path=checkpoint_path)

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\joecl\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:612: UserWarning: Checkpoint directory C:\Users\joecl\Documents(PC)\python\deep-learning\slitherio-rl\src\logs\Autoencoder-Experiment1\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
Restoring states from the checkpoint path at logs/Autoencoder-Experiment1/checkpoints/last-v1.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 166 K 
1 | decoder | Sequential | 94.1 K
---------------------------------------
260 K     Trainable params
0         Non-trainable params
260 K     Total params
0.520     Total estimated

Sanity Checking: 0it [00:00, ?it/s]

Training: 11it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]